In [47]:
# pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Auth at first time
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=132657856087-t2gdh3j2s3fcu1ugngr22ji2e7ksfs62.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
Modu

Authentication successful.


In [48]:
from __future__ import print_function
import json
import requests
import os
import os.path
import shutil
import errno
import sys
import urllib
import pandas as pd
from IPython.display import display

# facepy
import facepy

def gdrive_get_list():
    drive = GoogleDrive(gauth)

    # Auto-iterate through all files that matches this query
    file_list = drive.ListFile({'q': "'root' in parents"}).GetList()
    for file1 in file_list:
      print('title: %s, id: %s' % (file1['title'], file1['id']))

    # Paginate file lists by specifying number of max results
    for file_list in drive.ListFile({'maxResults': 10}):
      print('Received %s files from Files.list()' % len(file_list)) # <= 10
      for file1 in file_list:
        print('title: %s, id: %s' % (file1['title'], file1['id']))

def gdrive_upload_to_folder(fileName):
    # Create GoogleDrive instance with authenticated GoogleAuth instance.
    drive = GoogleDrive(gauth)

    # according to folder id
    f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": '1zlkUR9mitsedvx9trUjgxh54j9vua4go'}]})

    # Make sure to add the path to the file to upload below.
    f.SetContentFile('./downloads/'+fileName)
    f['title'] = fileName
    f.Upload()
    print('upload completed')

    # Insert the permission.
    permission = f.InsertPermission({
                            'type': 'anyone',
                            'value': 'anyone',
                            'role': 'reader'})

    #print(f['alternateLink'])
    return f['alternateLink']

def download_file(url, down):
    
    fileName = url.split('/')[4].split('?')[0]
    fileName = urllib.parse.unquote(fileName)
    
    if down:
        # write file
        r = requests.get(url, stream=True)
        with open('./downloads/'+fileName, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
                    f.flush()
        doctype = fileName.split('.')[-1]
    else:
        fileName = ''
        
    return fileName

def write_context(fileName, context):
    # write context to txt
    if context:
        with open('./downloads/'+fileName+'.txt', 'w') as f:
                f.write(context)
    

def dl_fb_files(groupId, accessToken):
    
    # Facebook API
    graph = facepy.GraphAPI(accessToken)
    getString = '/v3.0/' + groupId + '/files?fields=download_link,from,message'
    pages = graph.get(getString, page=True, retry=1)
    
    # Download Dir
    if os.path.isdir("./downloads"):
        shutil.rmtree('./downloads')
    os.mkdir('./downloads')
    
    df = pd.DataFrame(columns=['資源標題','年級','科目/單元','資源連結','doctype','內文'])

    for page in pages:
        for post in page['data']:
            
            # upload user 
            if 'from' in post:
                from_name = post['from']['name']
            else:
                confrom_nametext = ''
            
            # context
            if 'message' in post:
                context = post['message']
            else:
                context = ''
            
            # download link
            download_link = post['download_link']
            
            # download file
            fileName = download_file(download_link, 1)
            
            # Write context
            #write_context(fileName, context)
               
            #print('=====\n{}\n{}\n{}\n{}\n\n'.format(fileName, from_name, context, download_link))
            print(fileName)
            
            # upload to gdrive
            gdrive_link = gdrive_upload_to_folder(fileName)
            
            fileName_lst = fileName.split('.')
            title = fileName_lst[0]
            doctype = fileName_lst[-1]
            
            df = df.append({'資源標題':title,
                '年級':'',
                '科目/單元':'',
                '資源連結':gdrive_link,
                'doctype':doctype,
                '內文':context
                },ignore_index=True)
    display(df.head())
    df.to_csv('output.csv')

group_id = '1841438095921683'
usr_token = 'EAAUQuUxvNIUBAEn7Qp5zTNfN2GAPSrpm15m1LFsd0iaT6EhKdZAlZBvN8g0mGDdLUHfyFm15YfIwHpxi4VAkuWvVbU2rFZAKIEamZClYe4qgeCwKqPQlTL7NSJXG0dB7NO8XjygxKw55XyF5CfFqcOHAXb84IEAc5hNVUWNeDAZDZD'
dl_fb_files(group_id, usr_token)


upload completed
2-2.ppt
upload completed
1數學解題閱讀學習活動設計_南門  曾明德_國文的棋盤.docx


,資源標題,年級,科目/單元,資源連結,doctype,內文
0,2-2,,,https://drive.google.com/a/junyiacademy.org/fi...,ppt,這是內文這是內文這是內文這是內文這是內文這是內文這是內文\n123123123\nddddd
1,1數學解題閱讀學習活動設計_南門 曾明德_國文的棋盤,,,https://drive.google.com/a/junyiacademy.org/fi...,docx,


In [1]:
#https://lookaside.fbsbx.com/file/2-2.ppt?token=AWyebSAPL2lcDrxpSk_YYY2dmaHJPXKq2GDOiR520mkkXsIepoCjA3yvGW0KvfuF22qH1y9hVXrkGt2nMVdnJ8sn4OZgwTb5fyM4q42zmB90kmWvNJfnVB6Yrb3ZEI0u0Ux-InhBYZS7mURf6G3DHoozlpqkweXqKfavHonqCHfnok5YA6wYnLwKY0mFOmKzVcw
#https://lookaside.fbsbx.com/file/1%E6%95%B8%E5%AD%B8%E8%A7%A3%E9%A1%8C%E9%96%B1%E8%AE%80%E5%AD%B8%E7%BF%92%E6%B4%BB%E5%8B%95%E8%A8%AD%E8%A8%88_%E5%8D%97%E9%96%80%20%20%E6%9B%BE%E6%98%8E%E5%BE%B7_%E5%9C%8B%E6%96%87%E7%9A%84%E6%A3%8B%E7%9B%A4.docx?token=AWygxtrzMs5jC1AobwBoSqtgAG1pmiKjwS6IT3yU5OOLq9QghHAi99fGUqKBoW6qge6-zDfIQwiytszMsoHGPWBso0dceDRjFW7JsnCRZ0NLbjTXszgc43yXgZ9kAsksdW_Qi6k9-UnNtZBxSwViFn-MWNRq6iTHoBZMv5yMdeX-77jDRT9j47LZWrn6Gfo9N6E